In [1]:
cd ..

D:\project\FedImpute


D:\project\FedImpute\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import numpy as np
import loguru
import sys

## Step1 Prepare the data

We simulate a simple gaussain random data. Remember to make the `data_config` dictionary to be like the following, the keys in `data_config` should be consistent with the keys used for FedImpute.

In [3]:
data = np.random.rand(10000, 10)
data_config = {
    'task_type': 'regression',
    'clf_type': None,
    'num_cols': 9,
}

## Step2 Simulate Federated Missing Data Scenario

Import simulator and simulate the scanrios, details of parameters can be found in documentation

In [4]:
%load_ext autoreload
%autoreload 2
from fedimpute.simulator import Simulator
simulator = Simulator(debug_mode=False)
simulation_results = simulator.simulate_scenario(data, data_config, num_clients = 10, verbose=1)

Data partitioning...
Missing data simulation...
Simulation done. Using summary function to check the simulation results.


## Step3 Run the Federated Imputation

Run the federated imputation with the simulated data

In [6]:
%load_ext autoreload
%autoreload 2
from fedimpute.execution_environment import FedImputeEnv

env = FedImputeEnv()
env.configuration(imputer = 'gain', fed_strategy='fedavg', fit_mode = 'fed')
env.setup_from_simulator(simulator = simulator, verbose=1)
print(env.imputer_name, env.fed_strategy_name)

env.run_fed_imputation()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Setting up clients...
Setting up server...
Setting up workflow...
Environment setup complete.
gain fedavg
2024-06-25 22:05:50.915 | INFO     | fedimpute.execution_environment.workflows.workflow:eval_and_track:101 - 
Initial: rmse - 0.5785038752011208 ws - 0.28324699157925964


Global Epoch:   0%|          | 0/150 [00:00<?, ?it/s]

2024-06-25 22:06:21.376 | INFO     | fedimpute.execution_environment.workflows.workflow_jm:fed_imp_sequential:150 - All clients have converged. Stopping training at 40.
2024-06-25 22:06:21.377 | INFO     | fedimpute.execution_environment.workflows.workflow_jm:fed_imp_sequential:168 - start fine tuning ...
local True


Fine Tuning Epoch: 0it [00:00, ?it/s]

2024-06-25 22:06:23.173 | INFO     | fedimpute.execution_environment.workflows.workflow:eval_and_track:151 - Final: rmse - 0.28841122047809337 ws - 0.0897678714085901


## Step4 Evaluate the Imputation Outcome

Evaluate the imputation outcome, you can choose which of three evaluation aspects, you want to evaluate 

In [7]:
%load_ext autoreload
%autoreload 2
from fedimpute.evaluation import Evaluator

evaluator = Evaluator()
eval_ret = evaluator.evaluate(env, ['imp_quality', 'pred_downstream_local', 'pred_downstream_fed'])
evaluator.show_results()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Evaluating imputation quality...
Evaluating downstream prediction...


Clients:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating federated downstream prediction...


Global Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluation completed.
Evaluation Results
Imputation Quality             | rmse: 0.288 (0.00) nrmse: 0.710 (0.03) sliced-ws: 0.090 (0.01) 
Downstream Prediction (Local)  |  mse: 0.092 (0.01)   mae: 0.254 (0.01)      msle: 0.043 (0.01) 
Downstream Prediction (Fed)    |  mse: 0.088 (0.00)   mae: 0.253 (0.00)      msle: 0.041 (0.00) 
